# Use CellProfiler Features Extracted to Construct a knn Network

In [ ]:
import pandas

raw_data = pandas.read_csv(r"..\data\processed\cellprofiler_data.csv")
data = raw_data[["AreaOccupied", "Perimeter", "Image_Granularity_1_RescaleIntensity"]]
labels = raw_data["Image_Metadata_phenotype"]
data = data.to_numpy()
labels = labels.to_numpy()
phenotype = {"alveolar": 0,
             "marrow": 1,
             "monocyte": 2}
labels = [phenotype[label] for label in labels]

In [ ]:
from random import shuffle
train_index = [[] for _ in range(len(phenotype))] 
test_index = [[] for _ in range(len(phenotype))]

[train_index[labels[i]].append(i) for i in range(len(labels))]

for i in range(len(train_index)):
    tmp = train_index[i]
    shuffle(tmp)

    test_len = int(len(tmp)/5)
    test_index[i].extend(tmp[:test_len])
    train_index[i] = tmp[test_len:]

tmp = []
[tmp.extend(inds) for inds in train_index]
train_index = tmp

train_samples = [data[ind] for ind in train_index]
train_labels = [labels[ind] for ind in train_index]

tmp = []
[tmp.extend(inds) for inds in test_index]
test_index = tmp

test_samples = [data[ind] for ind in test_index]
test_labels = [labels[ind] for ind in test_index]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=len(phenotype))
knn.fit(train_samples, train_labels)


In [ ]:
out = list(knn.predict(test_samples))
correct = 0
total = len(out)
for pred, label in zip(out, test_labels):
    correct += int(pred == label)

accuracy = correct / total

print(f"accuracy of knn: {accuracy:>0.2%}")